#### The main purpose of this notebook is to convert the generated grey-scale masks into polygon labels that YOLO v8 needs.

## MOUNT DRIVE

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mask_dir = '/content/drive/MyDrive/aircraft_export_2/masks/'
label_dir = '/content/labels/'

In [ ]:
for split in ["train", "test"]:
    os.makedirs(os.path.join(label_dir, split), exist_ok = True)

In [ ]:
for split in ["train", "test"]:
    print(f"{os.path.basename(mask_dir)}/{split}:", len(os.listdir(os.path.join(mask_dir, split))))

## REFORMAT MASKS INTO POLYGON LABELS

In [ ]:
import cv2

In [ ]:
for split in ["train", "test"]:
    for j in os.listdir(os.path.join(mask_dir, split)):
        image_path = os.path.join(mask_dir, split, j)

        # load binary mask and get its contours
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # convert contours to polygons
        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 200:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)

        # print the polygons
        with open('{}.txt'.format(os.path.join(label_dir, split, j)[:-4]), 'w') as f:
            for polygon in polygons:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write('{}\n'.format(p))
                    elif p_ == 0:
                        f.write('0 {} '.format(p))
                    else:
                        f.write('{} '.format(p))

            f.close()

# [https://github.com/computervisioneng/image-segmentation-yolov8/blob/main/masks_to_polygons.py]

In [ ]:
!scp -r /content/labels "/content/drive/MyDrive/labels_2"